# Premissas do Algoritmo KNN (K-Nearest Neighbors)

O KNN é um algoritmo de aprendizado supervisionado baseado em instâncias. Ele classifica ou faz previsões com base nos K vizinhos mais próximos.

## Premissa 1: Similaridade entre Observações

**Princípio:** Observações similares tendem a ter comportamentos similares.

O KNN assume que pontos próximos no espaço de características compartilham a mesma classe ou valor.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Gerar dados sintéticos com 3 clusters
X, y = make_blobs(n_samples=100, centers=3, n_features=2, random_state=42)

# Visualizar
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', s=50, edgecolor='k')
plt.title('Pontos próximos tendem a pertencer à mesma classe')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)
plt.show()

## Premissa 2: Métrica de Distância Adequada

**Princípio:** A escolha da métrica de distância afeta diretamente a definição de "proximidade".

Principais métricas:
- **Euclidiana:** $d = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$ - Distância em linha reta
- **Manhattan:** $d = \sum_{i=1}^{n}|x_i - y_i|$ - Distância por caminhos ortogonais
- **Minkowski:** Generalização das anteriores

In [ ]:
# Comparação de métricas de distância
ponto_A = np.array([1, 1])
ponto_B = np.array([4, 5])

# Distância Euclidiana
dist_euclidiana = np.sqrt(np.sum((ponto_A - ponto_B)**2))

# Distância Manhattan
dist_manhattan = np.sum(np.abs(ponto_A - ponto_B))

print(f"Ponto A: {ponto_A}")
print(f"Ponto B: {ponto_B}")
print(f"\nDistância Euclidiana: {dist_euclidiana:.2f}")
print(f"Distância Manhattan: {dist_manhattan:.2f}")

# Visualizar as diferenças
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Euclidiana
ax[0].plot([ponto_A[0], ponto_B[0]], [ponto_A[1], ponto_B[1]], 'r-', linewidth=2, label='Euclidiana')
ax[0].scatter(*ponto_A, s=200, c='blue', edgecolor='k', zorder=3)
ax[0].scatter(*ponto_B, s=200, c='green', edgecolor='k', zorder=3)
ax[0].set_title('Distância Euclidiana')
ax[0].grid(True, alpha=0.3)
ax[0].legend()

# Manhattan
ax[1].plot([ponto_A[0], ponto_B[0]], [ponto_A[1], ponto_A[1]], 'b-', linewidth=2)
ax[1].plot([ponto_B[0], ponto_B[0]], [ponto_A[1], ponto_B[1]], 'b-', linewidth=2, label='Manhattan')
ax[1].scatter(*ponto_A, s=200, c='blue', edgecolor='k', zorder=3)
ax[1].scatter(*ponto_B, s=200, c='green', edgecolor='k', zorder=3)
ax[1].set_title('Distância Manhattan')
ax[1].grid(True, alpha=0.3)
ax[1].legend()

plt.tight_layout()
plt.show()

## Premissa 3: Normalização das Features

**Princípio:** Features em escalas diferentes podem dominar o cálculo de distância.

**Problema:** Uma feature com valores entre 0-1000 terá peso muito maior que uma feature entre 0-1.

**Solução:** Normalizar ou padronizar os dados antes de aplicar o KNN.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Dados com escalas diferentes
idade = np.array([25, 30, 35]).reshape(-1, 1)
salario = np.array([30000, 45000, 60000]).reshape(-1, 1)

# Combinar features
dados = np.hstack([idade, salario])

# Novo ponto para avaliar
ponto_novo = np.array([[28, 35000]])

# SEM normalização
dist_sem_norm = np.sqrt(np.sum((dados - ponto_novo)**2, axis=1))

# COM normalização
scaler = StandardScaler()
dados_norm = scaler.fit_transform(dados)
ponto_novo_norm = scaler.transform(ponto_novo)
dist_com_norm = np.sqrt(np.sum((dados_norm - ponto_novo_norm)**2, axis=1))

print("Dados originais:")
print("Idade | Salário")
for i in range(len(dados)):
    print(f"{dados[i, 0]:5.0f} | {dados[i, 1]:7.0f}")
print(f"\nPonto novo: [{ponto_novo[0, 0]:.0f}, {ponto_novo[0, 1]:.0f}]")

print("\n--- SEM Normalização ---")
print(f"Distâncias: {dist_sem_norm}")
print(f"Vizinho mais próximo: índice {np.argmin(dist_sem_norm)}")

print("\n--- COM Normalização ---")
print(f"Distâncias: {dist_com_norm}")
print(f"Vizinho mais próximo: índice {np.argmin(dist_com_norm)}")

print("\nNota: A diferença de escala faz o salário dominar o cálculo sem normalização.")

## Premissa 4: Escolha Adequada do K

**Princípio:** O valor de K afeta diretamente o viés e a variância do modelo.

- **K pequeno (ex: K=1):** Modelo mais complexo, sensível a ruídos (alta variância)
- **K grande:** Modelo mais simples, pode perder padrões locais (alto viés)
- **Regra prática:** K costuma ser ímpar para evitar empates em classificação binária

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Gerar dados sintéticos
X, y = make_blobs(n_samples=200, centers=3, n_features=2, random_state=42, cluster_std=1.5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Testar diferentes valores de K
valores_k = [1, 3, 5, 10, 20, 50]
acuracias = []

for k in valores_k:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    acuracias.append(acc)
    print(f"K={k:2d} -> Acurácia: {acc:.3f}")

# Visualizar impacto do K
plt.figure(figsize=(10, 5))
plt.plot(valores_k, acuracias, marker='o', linewidth=2, markersize=8)
plt.xlabel('Valor de K')
plt.ylabel('Acurácia')
plt.title('Impacto do K na Acurácia do Modelo')
plt.grid(True, alpha=0.3)
plt.xticks(valores_k)
plt.show()

## Premissa 5: Dados Balanceados

**Princípio:** Classes desbalanceadas podem enviesar as previsões.

Se uma classe tem muito mais exemplos que outra, o KNN tende a favorecer a classe majoritária. Em casos de desbalanceamento severo, considere:
- Reamostragem (oversampling/undersampling)
- Pesos diferentes para as classes
- Usar versões ponderadas do KNN

In [ ]:
from sklearn.datasets import make_classification
from collections import Counter

# Criar dataset desbalanceado (90% classe 0, 10% classe 1)
X_desb, y_desb = make_classification(n_samples=200, n_features=2, n_informative=2, 
                                      n_redundant=0, n_clusters_per_class=1,
                                      weights=[0.9, 0.1], random_state=42)

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_desb, y_desb, test_size=0.3, random_state=42)

print("Distribuição das classes no treino:")
print(Counter(y_train_d))

# KNN sem ponderação
knn_normal = KNeighborsClassifier(n_neighbors=5)
knn_normal.fit(X_train_d, y_train_d)
y_pred_normal = knn_normal.predict(X_test_d)

# KNN com ponderação por distância
knn_ponderado = KNeighborsClassifier(n_neighbors=5, weights='distance')
knn_ponderado.fit(X_train_d, y_train_d)
y_pred_ponderado = knn_ponderado.predict(X_test_d)

print("\n--- Resultados ---")
print(f"Acurácia KNN normal: {accuracy_score(y_test_d, y_pred_normal):.3f}")
print(f"Acurácia KNN ponderado: {accuracy_score(y_test_d, y_pred_ponderado):.3f}")

print("\nPrevisões KNN normal:")
print(Counter(y_pred_normal))
print("\nPrevisões KNN ponderado:")
print(Counter(y_pred_ponderado))

## Resumo das Premissas

| Premissa | Descrição | Ação |
|----------|-----------|------|
| **Similaridade** | Pontos próximos pertencem à mesma classe | Garantir features relevantes |
| **Métrica de distância** | Define o conceito de proximidade | Escolher métrica adequada ao problema |
| **Normalização** | Features em escalas diferentes distorcem distâncias | Sempre normalizar/padronizar |
| **Valor de K** | Define viés vs variância | Testar diferentes valores (validação cruzada) |
| **Balanceamento** | Classes desbalanceadas enviesam predições | Ponderar ou rebalancear dados |

**Quando usar KNN:**
- Dados com fronteiras de decisão não-lineares
- Datasets pequenos a médios
- Quando interpretabilidade local é importante

**Quando evitar KNN:**
- Datasets muito grandes (custo computacional alto)
- Muitas dimensões (curse of dimensionality)
- Dados com muito ruído